In [4]:
!pip install requests

from flask import Flask, jsonify, request
import requests
import time
import threading

app = Flask(__name__)

_cache = {}
_cache_lock = threading.Lock()
CACHE_TTL = 30

def get_from_cache(key):
    with _cache_lock:
        item = _cache.get(key)
        if not item:
            return None
        value, expiry = item
        if time.time() > expiry:
            del _cache[key]
            return None
        return value

def set_cache(key, value, ttl=CACHE_TTL):
    with _cache_lock:
        _cache[key] = (value, time.time() + ttl)

def fetch_joke_from_icanhaz():
    try:
        headers = {'Accept': 'application/json', 'User-Agent': 'Flask-Random-Joke/1.0'}
        r = requests.get('https://icanhazdadjoke.com/', headers=headers, timeout=5)
        if r.status_code == 200:
            data = r.json()
            return {'id': data.get('id'), 'type': 'dad', 'setup': None, 'punchline': data.get('joke')}
    except requests.RequestException:
        return None

def fetch_joke_from_official():
    try:
        r = requests.get('https://official-joke-api.appspot.com/jokes/random', timeout=5)
        if r.status_code == 200:
            data = r.json()
            return {'id': data.get('id'), 'type': data.get('type'), 'setup': data.get('setup'), 'punchline': data.get('punchline')}
    except requests.RequestException:
        return None

@app.route('/health', methods=['GET'])
def health_check():
    return jsonify({'status': 'ok'}), 200

@app.route('/joke', methods=['GET'])
def get_joke():
    cache_key = 'single_joke'
    cached = get_from_cache(cache_key)
    if cached:
        return jsonify({'source': 'cache', 'joke': cached}), 200
    joke = fetch_joke_from_icanhaz()
    if not joke:
        joke = fetch_joke_from_official()
    if not joke:
        return jsonify({'error': 'Failed to fetch joke from upstream providers'}), 503
    set_cache(cache_key, joke)
    return jsonify({'source': 'upstream', 'joke': joke}), 200

@app.route('/jokes', methods=['GET'])
def get_jokes():
    try:
        count = int(request.args.get('count', 5))
    except ValueError:
        return jsonify({'error': 'Invalid count parameter'}), 400
    if count < 1:
        return jsonify({'error': 'count must be >= 1'}), 400
    if count > 10:
        count = 10
    jokes = []
    try:
        r = requests.get('https://official-joke-api.appspot.com/random_ten', timeout=6)
        if r.status_code == 200:
            data = r.json()
            for item in data[:count]:
                jokes.append({'id': item.get('id'), 'type': item.get('type'), 'setup': item.get('setup'), 'punchline': item.get('punchline')})
    except requests.RequestException:
        pass
    while len(jokes) < count:
        j = fetch_joke_from_official() or fetch_joke_from_icanhaz()
        if not j:
            break
        jokes.append(j)
    if not jokes:
        return jsonify({'error': 'Failed to fetch jokes'}), 503
    return jsonify({'count': len(jokes), 'jokes': jokes}), 200

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)


Defaulting to user installation because normal site-packages is not writeable
 * Serving Flask app '__main__'
 * Debug mode: on



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.10.2:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1